In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [2]:
from sousvide.control.pilot import Pilot
import torch
import numpy as np

import sousvide.control.network_helper as nh

/home/admin/anaconda3/envs/kitchen/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.1'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
tcr = np.random.rand(1)
obj = np.random.rand(18)
icr = np.random.rand(360,640,3)
zcr = None

N = 10
xucr = 0.1*np.arange(15*N).reshape(15,N)
tcr = xucr[0,:]
xcr = xucr[1:11,:]
ucr = xucr[11:15,:]

print(xucr)

[[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]
 [ 1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9]
 [ 2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7  2.8  2.9]
 [ 3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9]
 [ 4.   4.1  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9]
 [ 5.   5.1  5.2  5.3  5.4  5.5  5.6  5.7  5.8  5.9]
 [ 6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9]
 [ 7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9]
 [ 8.   8.1  8.2  8.3  8.4  8.5  8.6  8.7  8.8  8.9]
 [ 9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7  9.8  9.9]
 [10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9]
 [11.  11.1 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9]
 [12.  12.1 12.2 12.3 12.4 12.5 12.6 12.7 12.8 12.9]
 [13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9]
 [14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9]]


In [ ]:
cohort = "general"

pilots = [
    Pilot(cohort,"Viper"),
    # Pilot(cohort,"Maverick"),
    # Pilot(cohort,"Iceman"),
    # Pilot(cohort,"Rooster"),
]

for pilot in pilots:
    upr = np.zeros(4)
    for i in range(10):
        unn,zcr,xnn,tsol = pilot.OODA(upr,tcr[i],xcr[:,i],obj,icr,zcr)
        upr = ucr[:,i]

        print(np.round(xnn['history'][0,:,0:14].cpu().numpy(), 2))
        print("=======================================================")

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
--------------------------------
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
--------------------------------
[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13. ]
 [ 0.1  1.1  2.1  3.1  4.1  5.1  6.1  7.1  8.1  9.1 10.1 11.1 12.1 13.1]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]]
--------------------------------
[[ 0.   0.   

In [ ]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [ ]:
# FiGS examples (scene_name, course_name)
cohort = "general"                        # cohort name
method,Nep_his,Nep_com = "alpha",5,5        # sv method,history epochs,command epochs

flights = [
    ('backroom','cluttered_env_track'),        # scene, course
    ]

roster = [
    "Maverick",                             # SV-Net
    ]

In [ ]:
# # Generate Rollouts
# rg.generate_rollout_data(cohort,method,flights)

# Review the Rollout Data
ps.plot_rollout_data(cohort)

In [ ]:
# Generate Observations
og.generate_observation_data(cohort,roster)

# Review the Observation Data
ps.plot_observation_data(cohort,roster)

In [ ]:
# Train the History Network
tp.train_roster(cohort,roster,"Parameter",Nep_his,lim_sv=10)

# Review Losses
pl.plot_losses(cohort,roster,"Parameter")

In [ ]:
# Train the Command Network
tp.train_roster(cohort,roster,"Commander",Nep_com,lim_sv=10)

# Review Losses
pl.plot_losses(cohort,roster,"Commander")

In [ ]:
# Simulate in FiGS
for scene,course in flights:
    df.simulate_roster(cohort,method,scene,course,roster)

# Plot the results
for _,course in flights:
    pl.review_simulations(cohort,course,roster,plot_show=True)